In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)

2017-05-18 14:00:38,859 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-05-18 14:00:38,861 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2017-05-18 14:00:38,862 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2017-05-18 14:00:38,870 : INFO : saved /tmp/deerwester.dict
2017-05-18 14:00:38,873 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2017-05-18 14:00:38,874 : INFO : saving sparse matrix to /tmp/deerwester.mm
2017-05-18 14:00:38,875 : INFO : PROGRESS: saving document #0
2017-05-18 14:00:38,877 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-05-18 14:00:38,886 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


In [5]:
from gensim import corpora, models, similarities

In [8]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

2017-05-18 14:01:44,381 : INFO : collecting document frequencies
2017-05-18 14:01:44,382 : INFO : PROGRESS: processing document #0
2017-05-18 14:01:44,383 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [9]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf]

2017-05-18 14:01:45,674 : INFO : using serial LSI version on this node
2017-05-18 14:01:45,675 : INFO : updating model with new documents
2017-05-18 14:01:45,676 : INFO : preparing a new chunk of documents
2017-05-18 14:01:45,678 : INFO : using 100 extra samples and 2 power iterations
2017-05-18 14:01:45,679 : INFO : 1st phase: constructing (12, 102) action matrix
2017-05-18 14:01:45,682 : INFO : orthonormalizing (12, 102) action matrix
2017-05-18 14:01:45,690 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-05-18 14:01:45,694 : INFO : computing the final decomposition
2017-05-18 14:01:45,695 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-05-18 14:01:45,696 : INFO : processed documents up to #9
2017-05-18 14:01:45,698 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-05-18 14:01:45,699 : INFO : topic #

In [10]:
lsi.print_topics(2)

2017-05-18 14:02:01,516 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-05-18 14:02:01,517 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [11]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, 0.066007833960901346), (1, -0.52007033063618502)]
[(0, 0.19667592859142263), (1, -0.76095631677000541)]
[(0, 0.089926399724461425), (1, -0.72418606267525132)]
[(0, 0.075858476521778906), (1, -0.63205515860034334)]
[(0, 0.10150299184979988), (1, -0.57373084830029608)]
[(0, 0.70321089393783187), (1, 0.16115180214025573)]
[(0, 0.87747876731198415), (1, 0.16758906864659137)]
[(0, 0.9098624686818586), (1, 0.14086553628718729)]
[(0, 0.61658253505692817), (1, -0.053929075663895709)]


### thought experiment
* say we have 5 docs and 12 words, the DW matrix is 5x12
* LSI with 2 latent topics creates a 5x2 doc matrix and 2x12 word matrix
* 5x2 is the topic distribution of the 5 topics